In [1]:
from datetime import datetime, timedelta, timezone

SHA_TZ = timezone(
  timedelta(hours=8),
  name='Asia/Shanghai',
)
utc_now = datetime.utcnow().replace(tzinfo=timezone.utc)
beijing_now = utc_now.astimezone(SHA_TZ)
beijing_now.strftime('%Y-%m-%d %H:%M:%S')

'2024-11-30 11:36:34'

In [2]:
import sys, os
import json
def is_kaggle_interactive():
  if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    return os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
  return True

In [3]:
!cp /kaggle/input/aslfr-model/* ./

In [4]:
!cat ./inference_args.json

{"selected_columns": ["x_left_hand_0", "y_left_hand_0", "z_left_hand_0", "x_left_hand_1", "y_left_hand_1", "z_left_hand_1", "x_left_hand_2", "y_left_hand_2", "z_left_hand_2", "x_left_hand_3", "y_left_hand_3", "z_left_hand_3", "x_left_hand_4", "y_left_hand_4", "z_left_hand_4", "x_left_hand_5", "y_left_hand_5", "z_left_hand_5", "x_left_hand_6", "y_left_hand_6", "z_left_hand_6", "x_left_hand_7", "y_left_hand_7", "z_left_hand_7", "x_left_hand_8", "y_left_hand_8", "z_left_hand_8", "x_left_hand_9", "y_left_hand_9", "z_left_hand_9", "x_left_hand_10", "y_left_hand_10", "z_left_hand_10", "x_left_hand_11", "y_left_hand_11", "z_left_hand_11", "x_left_hand_12", "y_left_hand_12", "z_left_hand_12", "x_left_hand_13", "y_left_hand_13", "z_left_hand_13", "x_left_hand_14", "y_left_hand_14", "z_left_hand_14", "x_left_hand_15", "y_left_hand_15", "z_left_hand_15", "x_left_hand_16", "y_left_hand_16", "z_left_hand_16", "x_left_hand_17", "y_left_hand_17", "z_left_hand_17", "x_left_hand_18", "y_left_hand_18", 

In [5]:
len(json.load(open('./inference_args.json'))['selected_columns'])

384

In [6]:
try:
  from icecream import ic
except Exception:
  !pip install /kaggle/input/tflite-runtime-nightly-2140dev20240602-whl/tflite_runtime_nightly-2.14.0.dev20240602-cp310-cp310-manylinux2014_x86_64.whl
  !pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream/ 

Processing /kaggle/input/tflite-runtime-nightly-2140dev20240602-whl/tflite_runtime_nightly-2.14.0.dev20240602-cp310-cp310-manylinux2014_x86_64.whl


In [7]:
#if is_kaggle_interactive():
from icecream import ic
import tflite_runtime.interpreter as tflite
import tflite_runtime
ic(tflite_runtime.__version__)
import time
import json
import pandas as pd
from tqdm.auto import tqdm
from leven import levenshtein
# import tensorflow as tf
import numpy as np

model_dir = './'
fold = 0
total = 20
total = 50

SEL_FEATURES = json.load(
    open(f'{model_dir}/inference_args.json'))['selected_columns']
# ic(SEL_FEATURES)

def load_relevant_data_subset(pq_path):
  return pd.read_parquet(pq_path, columns=SEL_FEATURES)  #selected_columns)

with open("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json",
          "r") as f:
  character_map = json.load(f)
rev_character_map = {j: i for i, j in character_map.items()}

df = pd.read_csv('/kaggle/input/aslfr-input/train.csv')
df = df[df.fold == fold]
df = df.reset_index(drop=True)

idx = 0
sample = df.loc[idx]
loaded = load_relevant_data_subset('/kaggle/input/asl-fingerspelling/' +
                                   sample['path'])
loaded = loaded[loaded.index == sample['sequence_id']].values
print(loaded.shape)
frames = loaded

interpreter = tflite.Interpreter(f'{model_dir}/model.tflite')
# interpreter = tf.lite.Interpreter(f'{model_dir}/model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())

REQUIRED_SIGNATURE = 'serving_default'
REQUIRED_OUTPUT = 'outputs'
if REQUIRED_SIGNATURE not in found_signatures:
  raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output_lite = prediction_fn(inputs=frames)
prediction_str = "".join([
    rev_character_map.get(s, "")
    for s in np.argmax(output_lite[REQUIRED_OUTPUT], axis=1)
])
ic(sample['phrase'], prediction_str)

st = time.time()
cnt = 0
model_time = 0
ic(total, len(df))
total = min(len(df), total)

l = []
for i in tqdm(range(total)):
  sample = df.loc[i]
  loaded = load_relevant_data_subset('/kaggle/input/asl-fingerspelling/' +
                                     sample['path'])
  loaded = loaded[loaded.index == sample['sequence_id']].values
  # ic(loaded)
  md_st = time.time()
  output_ = prediction_fn(inputs=loaded)
  model_time += time.time() - md_st

  prediction_str = "".join([
      rev_character_map.get(s, "")
      for s in np.argmax(output_[REQUIRED_OUTPUT], axis=1)
  ])
  distance = levenshtein(sample['phrase'], prediction_str)
  phrase_true = sample['phrase']
  phrase_pred = prediction_str
  l.append({
    'sequence_id': sample['sequence_id'],
    'phrase_true': phrase_true,
    'phrase_pred': phrase_pred,
    'phrase_len_true': len(phrase_true),
    'phrase_len_pred': len(phrase_pred),
    'distance': distance,
    'score': max(len(phrase_true) - distance, 0.) / len(phrase_true),
  })

df = pd.DataFrame(l)
score = max(df['phrase_len_true'].sum() - df['distance'].sum(), 0.) / df['phrase_len_true'].sum()
display(df.head(20))
mean_time = model_time / total
print(beijing_now.strftime('%Y-%m-%d %H:%M:%S'))
print(open('./path.txt').readline().split('working/')[1].split('/ckpt')[0])
os.system('du -h ./model.tflite')
import pandas as pd
metrics = pd.read_csv('./metrics.csv').iloc[-1]
print('score:', metrics['score'])
print('score/head', metrics['score/head'])
print('tflite_score/head:', score)
print('mean_time:', mean_time)

ic| tflite_runtime.__version__: '2.14.0dev20240602'


(56, 384)


ic| sample['phrase']: '242-197-6202', prediction_str: '2617-602'
ic| total: 50, len(df): 15045


  0%|          | 0/50 [00:00<?, ?it/s]

,sequence_id,phrase_true,phrase_pred,phrase_len_true,phrase_len_pred,distance,score
0,1817195757,242-197-6202,2617-602,12,8,5,0.583333
1,1817282569,+51-2721-211-63,+51-2721-211-63,15,15,0,1.000000
2,1817348598,wildberries_ru,wildberes_r,14,11,3,0.785714
3,1817370426,projecteur-led,euaa,14,4,12,0.142857
4,1817672186,596-033-4046,+576-038-4044,12,13,4,0.666667
5,1817721159,tampa fl,tampa fl,8,8,0,1.000000
6,1817740422,492288 west 28th terrace south,492288 west 28th terrace south,30,30,0,1.000000
7,1818613605,m-advice.co.jp/boosela,maco/,22,5,17,0.227273
8,1818712825,6169 valley view parks,6197 lideli paks,22,16,12,0.454545
9,1818787239,neobychnye-muzei-mira/3585/,neobychnye-muzei-mira/3585/,27,27,0,1.000000


2024-11-27 13:48:00
online/30/0/final-20layers.finetune.1127.eval
39M	./model.tflite
score: 0.87053
score/head 0.806283
tflite_score/head: 0.8270142180094787
mean_time: 0.9261609554290772
